# 说明


KNN是一种非常经典的非参数学习算法，相比于基于最小二乘的线性回归算法，KNN也可以看做一种基于X邻域的条件估计

In [46]:
import functools as func
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets as ds

%matplotlib inline

In [68]:
@func.total_ordering
class BinaryTreeNode:
    def __init__(self, value, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __lt__(self, node):
        return self.value < node.value
    
    def __eq__(self, node):
        return self.value == node.value
    
    def __str__(self):
        return str(self.value)
    
    def has_right_child(self):
        return self.right is not None
    
    def has_left_child(self):
        return self.left is not None
    
    def has_child(self):
        return self.has_right_child() or self.has_left_child()

In [54]:
BinaryTreeNode(2) is None

False

In [69]:
class BinaryTree:
    def __init__(self):
        self.root = None
    
    def add(self, value):
        node = BinaryTreeNode(value)
        if not self.is_empty():
            self._add(self.root, node)
        else:
            self.root = node

    
    def _add(self, parent, node_new):
        if node_new < parent:
            print('Insert into left leaf')
            if parent.has_left_child():
                self._add(parent.left, node_new)
            else:
                parent.left = node_new
        else:
            print('Insert into right leaf')
            if parent.has_right_child():
                self._add(parent.right, node_new)
            else:
                parent.right = node_new
        
    
    def clear(self):
        if not self.is_empty():
            self.root = None
            
    def is_empty(self):
        return self.root is None
    
    def find(self, value):
        if self.is_empty():
            return False
        
        return self._find(self.root, value)
        
    
    def _find(self, node, value):
        if node.value == value:
            return True
        elif value < node.value:
            return not has_left_child() or self._find(node.left, value)

        else:
            return not has_right_child() or self._find(node.right, value)

    
    def print_tree(self):
        if self.root:
            self._print_tree(self.root)
    
    def _print_tree(self, node):
        if node is not None:
            self._print_tree(node.left)
            print(str(node.value) + ' ')
            self._print_tree(node.right)

In [28]:
bst = BinaryTree()

In [29]:
bst.add(1)

In [30]:
bst.add(5)
bst.add(2)

Insert into right leaf
Insert into right leaf
Insert into left leaf


In [32]:
bst.add(7)
bst.add(3)
bst.add(6)
bst.add(4)

Insert into right leaf
Insert into right leaf
Insert into right leaf
Insert into left leaf
Insert into right leaf
Insert into right leaf
Insert into right leaf
Insert into left leaf
Insert into right leaf
Insert into left leaf
Insert into right leaf
Insert into right leaf


In [33]:
bst.print_tree()

1 
2 
3 
4 
5 
6 
7 


In [70]:
tree = BinaryTree()
tree.add(3)
tree.add(4)
tree.add(0)
tree.add(8)
tree.add(2)

Insert into right leaf
Insert into left leaf
Insert into right leaf
Insert into right leaf
Insert into left leaf
Insert into right leaf


In [71]:
tree.print_tree()

0 
2 
3 
4 
8 


In [17]:
knn = KNNClassifier()

In [18]:
knn.n_neighbors

2

In [20]:
knn.kdtree()

Begin to build kdtree model
